In [38]:
#Inception Module


# Load pickled data
import pickle
import numpy as np
import tensorflow as tf 

# TODO: Fill this in based on where you saved the training and testing data

training_file = "train_gray_contrast.p"
testing_file = "test_final_gray.p"
validation_file = "validation_gray_contrast.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
# with open(testing_file, mode='rb') as f:
#     test = pickle.load(f)
with open(validation_file, mode='rb') as f:
    validation = pickle.load(f)
    
X_train_old, y_train = train['features'], train['labels']
# X_test, y_test = test['features'], test['labels']
X_validation_old, y_validation = validation['features'], validation['labels']


In [39]:
from sklearn.preprocessing import normalize

X_new = X_train_old#.astype(float)
X_new = X_new.astype(float)
X_new-=128.0
X_train = X_new /255.0

X_val = X_validation_old#.astype(float)
X_val = X_val.astype(float)
X_val-=128.0
X_validation = X_val /255.0

# X_test = X_test#.astype(float)
# X_test = X_test.astype(float)
# X_test-=128.0
# X_test = X_test /255.0

In [40]:

from tensorflow.contrib.layers import flatten
import tensorflow as tf


def conv2d(x,weight,bias,strides=1,padding='SAME'):
    layer = tf.nn.conv2d(x,weight,[1,strides,strides,1],padding=padding)
    layer = tf.nn.bias_add(layer,bias)
    layer = tf.nn.relu(layer)
    return layer

def maxpool(x,k=2,padding='SAME',stride=2):
    kernel_size = [1,k,k,1]
    return tf.nn.max_pool(x,ksize=kernel_size,strides=[1,stride,stride,1],padding=padding)
    

In [41]:

#https://hacktilldawn.com/2016/09/25/inception-modules-explained-and-implemented/
def naive_inception(x):
    
    mu = 0
    sigma = 0.1
    in_depth = x.get_shape().as_list()[3]
    layer_depth = {
    'layer_1_1x1': 3,
    'layer_1_3x3': 3,   
    'layer_1_5x5': 3
    }
    
    #1x1 conv
    weights_1x1 = tf.Variable(tf.random_normal([1, 1,in_depth , layer_depth['layer_1_1x1']], mean=0.0, stddev=1.0, dtype=tf.float32) )
    bias_1x1 = tf.Variable(tf.zeros(layer_depth['layer_1_1x1']))
    
    conv_1x1 = conv2d(x,weights_1x1,bias_1x1)
    print(conv_1x1.get_shape())
    
    #3x3 conv
    weights_3x3 = tf.Variable(tf.random_normal([3, 3,in_depth , layer_depth['layer_1_3x3']], mean=0.0, stddev=1.0, dtype=tf.float32) )
    bias_3x3 = tf.Variable(tf.zeros(layer_depth['layer_1_3x3']))
    
    conv_3x3 = conv2d(x,weights_3x3,bias_3x3)
    print(conv_3x3.get_shape())

    #5x5 conv
    weights_5x5 = tf.Variable(tf.random_normal([5, 5,in_depth , layer_depth['layer_1_5x5']], mean=0.0, stddev=1.0, dtype=tf.float32) )
    bias_5x5 = tf.Variable(tf.zeros(layer_depth['layer_1_5x5']))
    
    conv_5x5 = conv2d(x,weights_5x5,bias_5x5)
    print(conv_5x5.get_shape())

    #3x3 maxpooling
    
    maxpool_3x3 = maxpool(x,k=3,stride=1)
    print(maxpool_3x3.get_shape())

    #stack 
    incepction_tensor = tf.concat(3,(conv_1x1,conv_3x3,conv_5x5,maxpool_3x3))
    
    return incepction_tensor

def fully_connected(in_tensor,layers,keep_prob):
    mu = 0
    sigma = 0.1
    in_dim = np.product(in_tensor.get_shape().as_list()[1:])
    tensor_flatten = flatten(in_tensor)
    
    for each in layers:
        weights = tf.Variable(tf.random_normal([in_dim, each],mu,sigma,dtype=tf.float32))
        bias =  tf.Variable(tf.zeros(each))

        fc1_drop = tf.nn.dropout(tensor_flatten, keep_prob)    
        fc2 = tf.add(
        tf.matmul(fc1_drop, weights),
        bias)
        # TODO: Activation.
        tensor_flatten = tf.nn.tanh(fc2)
        in_dim = each
    
    return tensor_flatten
         
        
    


In [42]:
INPUT_DEPTH = 1
tf.reset_default_graph()
x = tf.placeholder(tf.float32, (None, 32, 32, INPUT_DEPTH))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)
incep_tensor = naive_inception(x)
logits = fully_connected(incep_tensor,[90,120,43],keep_prob)

(?, 32, 32, 3)
(?, 32, 32, 3)
(?, 32, 32, 3)
(?, 32, 32, 1)


In [43]:
rate = 0.0005
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
# Gradient clipping
gvs = optimizer.compute_gradients(loss_operation)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
optimizer.apply_gradients(capped_gvs)

training_operation = optimizer.minimize(loss_operation)

In [44]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [45]:
EPOCHS = 100
BATCH_SIZE = 128

In [69]:
from sklearn.utils import shuffle
from imblearn.over_sampling import RandomOverSampler
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
model_file = "./inception_naive_gray_v2"
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    num_examples =len(X_train)#392090
    from pathlib import Path
    

    my_file = Path(model_file)
    if my_file.is_file():
        saver.restore(sess,model_file)
        print("Model restored...")
    print("Training...")
    print()
    for i in range(EPOCHS):
        ros = RandomOverSampler()
        x_index = np.arange(0,len(y_train),1)
        x_index_resampled, y_resampled = ros.fit_sample(np.expand_dims(x_index,axis=1), y_train)
        x_in = np.squeeze(x_index_resampled)
        X_resampled = X_train[x_in,:,:,]
        X_train, y_train = shuffle(X_resampled, y_resampled)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:1.0})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        training_accuracy = evaluate(batch_x, batch_y)
        mean_cross_entropy = sess.run(loss_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:1.0})
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Cross entropy = {:.4f}".format(mean_cross_entropy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        
    saver.save(sess, model_file)
    print("Model saved")

Training...

EPOCH 1 ...
Training Accuracy = 0.711
Cross entropy = 2.4268
Validation Accuracy = 0.557
EPOCH 2 ...
Training Accuracy = 0.859
Cross entropy = 2.2212
Validation Accuracy = 0.679
EPOCH 3 ...
Training Accuracy = 0.914
Cross entropy = 2.0912
Validation Accuracy = 0.749
EPOCH 4 ...
Training Accuracy = 0.953
Cross entropy = 2.0355
Validation Accuracy = 0.772
EPOCH 5 ...
Training Accuracy = 0.922
Cross entropy = 2.0191
Validation Accuracy = 0.793
EPOCH 6 ...
Training Accuracy = 0.961
Cross entropy = 1.9840
Validation Accuracy = 0.806
EPOCH 7 ...
Training Accuracy = 0.984
Cross entropy = 1.9476
Validation Accuracy = 0.822
EPOCH 8 ...
Training Accuracy = 0.977
Cross entropy = 1.9473
Validation Accuracy = 0.823


KeyboardInterrupt: 